In [1]:
# In src/backbone.py
from mindspore import nn
from mindcv.models import create_model

def create_efficientnet_backbone():
    """
    Loads a pre-trained EfficientNet-B1 model and prepares it
    to be used as a backbone for an object detection network.
    """
    print("Loading pre-trained EfficientNet-B1 backbone...")
    
    # 1. Load the pre-trained classification model from mindcv
    backbone = create_model('efficientnet_b1', pretrained=True)
    
    # 2. Remove the final classification layer (the "head")
    #    We replace it with an Identity layer that does nothing.
    #    This exposes the powerful features from the main body of the network.
    backbone.head = nn.Identity()
    
    print("Backbone created successfully!")
    return backbone

In [ ]:
# In main.py
import mindspore as ms
from mindspore.dataset import GeneratorDataset
from mindspore.dataset.transforms import TypeCast
# --- IMPORT THESE NEW TRANSFORMS ---
import mindspore.dataset.vision as vision

# Import your custom dataset loader and new backbone creator
from src.coco_dataset import CocoDataset
from src.backbone import create_efficientnet_backbone

# Set the context to CPU
ms.set_context(device_target="CPU")

## 1. Prepare your COCO Dataset
print("Initializing COCO dataset...")
annotations_path = './coco2017/annotations/instances_train2017.json'
images_path = './coco2017/train2017'
coco_ds = CocoDataset(annotations_file=annotations_path, images_dir=images_path)

dataset = GeneratorDataset(
    source=coco_ds,
    column_names=["image", "boxes", "labels"],
    shuffle=True
)

# --- THIS IS THE FIX ---
# Define a list of transformations to apply to each image.
# The order is important.
transforms = [
    vision.Resize((224, 224)),      # 1. Resize the image to 224x224
    TypeCast(ms.float32),           # 2. Convert the data type to float
    vision.HWC2CHW()                # 3. Change shape from (H, W, C) to (C, H, W)
]

# Apply the transformations to the 'image' column.
dataset = dataset.map(operations=transforms, input_columns="image")


# Now that images are a fixed size, you can batch them.
dataset = dataset.batch(batch_size=1) # Using batch size of 1 for this test

## 2. Create your pre-trained backbone
backbone_model = create_efficientnet_backbone()

## 3. Test the flow of data
print("\n--- Testing Data Flow ---")

# Get one batch of data from your COCO loader
data_iterator = dataset.create_dict_iterator()
data_batch = next(data_iterator)

image_tensor = data_batch['image']
ground_truth_boxes = data_batch['boxes']

print(f"Loaded a preprocessed image with shape: {image_tensor.shape}") # Should be (1, 3, 224, 224)

# 4. Pass the image through the backbone to get features
#    In a real model, these features would go to the detection head.
feature_map = backbone_model(image_tensor)

print(f"Backbone produced a feature map with shape: {feature_map.shape}")
print("\nThis feature map is what an object detection head would use to find bounding boxes.")

[WARNING] ME(2872:2020,MainProcess):2025-10-12-20:42:58.347.000 [mindspore\context.py:1412] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.


Initializing COCO dataset...
Initializing COCO dataset loader...
Dataset initialized with 118287 images.
Loading pre-trained EfficientNet-B1 backbone...
Backbone created successfully!

--- Testing Data Flow ---
Loaded a preprocessed image with shape: (1, 3, 224, 224)
Backbone produced a feature map with shape: (1, 1000)

This feature map is what an object detection head would use to find bounding boxes.


In [6]:
# In main.py
import mindspore as ms
from mindspore.dataset import GeneratorDataset
from mindspore.dataset.transforms import TypeCast
import mindspore.dataset.vision as vision

# Import your custom dataset loader and the FINAL detector model
from src.coco_dataset import CocoDataset
from src.detector import TrafficVisionDetector  # <-- IMPORT THE NEW, COMPLETE MODEL

# Set the context to CPU
ms.set_context(device_target="CPU",pynative_synchronize=True) # Added pynative_synchronize for easier debugging

## 1. Prepare your COCO Dataset
print("Initializing COCO dataset...")
annotations_path = './coco2017/annotations/instances_train2017.json'
images_path = './coco2017/train2017'
coco_ds = CocoDataset(annotations_file=annotations_path, images_dir=images_path)

dataset = GeneratorDataset(
    source=coco_ds,
    column_names=["image", "boxes", "labels"],
    shuffle=True
)

# Define the image transformation pipeline
transforms = [
    vision.Resize((224, 224)),
    TypeCast(ms.float32),
    vision.HWC2CHW()
]

# Apply the transformations to the 'image' column.
dataset = dataset.map(operations=transforms, input_columns="image")

# Now that images are a fixed size, you can batch them.
dataset = dataset.batch(batch_size=1) # Using batch size of 1 for this test

## 2. Create your FULL object detection model
#    Your project has 8 traffic classes.
print("\nCreating the full TrafficVisionDetector model...")
model = TrafficVisionDetector(num_classes=8)
model.set_train(False) # Set to evaluation mode for this test

## 3. Test the flow of data through the entire model
print("\n--- Testing Full Model ---")
data_iterator = dataset.create_dict_iterator()
data_batch = next(data_iterator)
image_tensor = data_batch['image']

print(f"Loaded a preprocessed image with shape: {image_tensor.shape}")

# 4. Pass the image through the full model to get final predictions
box_predictions, class_predictions = model(image_tensor)

print(f"\nModel produced Box Predictions with shape: {box_predictions.shape}")
print(f"Model produced Class Predictions with shape: {class_predictions.shape}")
print("\nThese are the final outputs you will use in a loss function to train the model!")

[WARNING] ME(2872:2020,MainProcess):2025-10-12-20:58:18.725.000 [mindspore\context.py:1412] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.
[WARNING] ME(2872:2020,MainProcess):2025-10-12-20:58:18.725.000 [mindspore\context.py:1412] For 'context.set_context', the parameter 'pynative_synchronize' will be deprecated and removed in a future version. Please use the api mindspore.runtime.launch_blocking() instead.


Initializing COCO dataset...
Initializing COCO dataset loader...
Dataset initialized with 118287 images.

Creating the full TrafficVisionDetector model...
Loading pre-trained EfficientNet-B1 backbone...
Backbone created successfully!

--- Testing Full Model ---
Loaded a preprocessed image with shape: (1, 3, 224, 224)


ValueError: For primitive[Conv2D], the x shape size must be equal to 4, but got 2.

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore\core\utils\check_convert_utils.cc:626 mindspore::CheckAndConvertUtils::CheckInteger
